In [6]:
%pip install lpips
%pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 362.4/664.8 MB 6.3 MB/s eta 0:00:48
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.1 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please up

## Download ConreolNet models

In [34]:
# Clean previous attempts
if os.path.exists('/content/drive/MyDrive/models'):
    shutil.rmtree('/content/drive/MyDrive/models')

# 2. MANUAL DOWNLOAD FUNCTION
def download_controlnet(model_name, model_dir):
    model_path = f"/content/drive/MyDrive/models/controlnet/{model_name}"
    if not os.path.exists(model_path):
        os.makedirs(model_path, exist_ok=True)

        # Download essential files
        !wget -q https://huggingface.co/lllyasviel/{model_name}/resolve/main/config.json -P {model_path}
        !wget -q https://huggingface.co/lllyasviel/{model_name}/resolve/main/diffusion_pytorch_model.safetensors -P {model_path}

        # Verify download
        if not os.path.exists(f"{model_path}/config.json"):
            raise FileNotFoundError(f"config.json missing for {model_name}")
        if not os.path.exists(f"{model_path}/diffusion_pytorch_model.safetensors"):
            raise FileNotFoundError(f"Model file missing for {model_name}")

        print(f"✓ Successfully downloaded {model_name}")
    return model_path

# 3. DOWNLOAD SPECIFIC MODELS
try:
    # Download ControlNet models (using verified working versions)
    cn_models = {
        "inpaint": "control_v11p_sd15_inpaint",
        "canny": "control_v11p_sd15_canny",
        "seg": "control_v11p_sd15_seg"
    }

    for name, model_id in cn_models.items():
        download_controlnet(model_id, f"/content/drive/MyDrive/models/controlnet/{name}")

except Exception as e:
    print(f"❌ Download failed: {e}")
    raise

✓ Successfully downloaded control_v11p_sd15_inpaint
✓ Successfully downloaded control_v11p_sd15_canny
✓ Successfully downloaded control_v11p_sd15_seg


In [36]:
# 4. VERIFY AND LOAD MODELS
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline
import torch

def load_model(model_class, model_path):
    try:
        return model_class.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            local_files_only=True
        )
    except Exception as e:
        print(f"❌ Failed to load {model_path}: {str(e)[:200]}")
        raise

try:
    # Load base model
    # pipe = load_model(StableDiffusionInpaintPipeline, "/content/drive/MyDrive/models/stable-diffusion")
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting",
        torch_dtype=torch.float16
    )

    # Load ControlNets
    controlnets = {}
    for name in cn_models.keys():
        model_path = f"/content/drive/MyDrive/models/controlnet/{name}"
        controlnet = load_model(ControlNetModel, model_path)
        controlnets[name] = StableDiffusionControlNetInpaintPipeline(
            vae=pipe.vae,
            text_encoder=pipe.text_encoder,
            tokenizer=pipe.tokenizer,
            unet=pipe.unet,
            scheduler=pipe.scheduler,
            safety_checker=pipe.safety_checker,
            feature_extractor=pipe.feature_extractor,
            controlnet=controlnet
        )

    print("✅ All models loaded successfully!")
    print("Available ControlNets:", list(controlnets.keys()))
except Exception as e:
    print(f"❌ Model loading failed: {e}")
    raise

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the p

✅ All models loaded successfully!
Available ControlNets: ['inpaint', 'canny', 'seg']


In [1]:
from google.colab import drive
import numpy as np
import lpips
import os
import csv
from glob import glob
from tqdm import tqdm
from PIL import Image
import torch
from concurrent.futures import ThreadPoolExecutor
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline
import random
import shutil
from torchvision import transforms
from skimage.metrics import peak_signal_noise_ratio as compute_psnr
from skimage.metrics import structural_similarity as compute_ssim

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

### Set seeds for reproducibility

In [7]:
# ---------------------------
# Set seeds for reproducibility
# ---------------------------
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

### LPIPS model loading

In [8]:
# ---------------------------
# LPIPS model loading
# ---------------------------
def load_lpips_model(model_dir="models/lpips"):
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, "lpips_alex.pth")
    model = lpips.LPIPS(net='alex')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
    else:
        torch.save(model.state_dict(), model_path)
    model.eval()
    if torch.cuda.is_available():
        model.cuda()
    return model

lpips_model = load_lpips_model()

NameError: name 'lpips' is not defined

### Evaluation functions

In [ ]:
# ---------------------------
# Evaluation functions
# ---------------------------
def prepare_for_lpips(pil_image):
    tensor = transforms.ToTensor()(pil_image).unsqueeze(0)
    tensor = tensor * 2 - 1
    if torch.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

def evaluate_metrics(gt_img, inpaint_img):
    gt_np = np.array(gt_img).astype(np.float32) / 255.0
    inpaint_np = np.array(inpaint_img).astype(np.float32) / 255.0

    if gt_np.shape != inpaint_np.shape:
        inpaint_img = inpaint_img.resize(gt_img.size, Image.Resampling.LANCZOS)
        inpaint_np = np.array(inpaint_img).astype(np.float32) / 255.0

    psnr = compute_psnr(gt_np, inpaint_np, data_range=1.0)

    min_size = min(gt_np.shape[0], gt_np.shape[1])
    win_size = 7 if min_size >= 7 else (min_size if min_size % 2 == 1 else min_size - 1)
    ssim = compute_ssim(gt_np, inpaint_np, win_size=win_size, channel_axis=2, data_range=1.0)

    gt_tensor = prepare_for_lpips(gt_img)
    inpaint_tensor = prepare_for_lpips(inpaint_img)
    with torch.no_grad():
        lpips_distance = lpips_model(gt_tensor, inpaint_tensor).item()

    return psnr, ssim, lpips_distance


### ControlNet helper functions

In [ ]:
def run_controlnet_inpaint(image_path, mask_path, fake_path, pipe, reference_images, prompt, output_path, seed=42):
    # print(f"Starting inpainting for: {os.path.basename(image_path)}")

    # Load original images
    image = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")
    control_image = Image.open(fake_path).convert("RGB")

    # Store original size for later resizing
    original_size = image.size

    # Target resolution for speed-up
    TARGET_RES = (512, 512)
    # print("Resizing images to 512x512 for faster processing...")
    image = image.resize(TARGET_RES, Image.Resampling.LANCZOS)
    mask = mask.resize(TARGET_RES, Image.Resampling.LANCZOS)
    control_image = control_image.resize(TARGET_RES, Image.Resampling.LANCZOS)

    generator = torch.manual_seed(seed)

    # Run pipeline
    # print("Running ControlNet pipeline...")
    result = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask,
        control_image=control_image,
        num_inference_steps=15, # can be increased for better performa
        generator=generator
    ).images[0]

    # Optional: Resize back to original resolution
    # print("Resizing result back to original resolution...")
    result = result.resize(original_size, Image.Resampling.LANCZOS)

    # Save output
    print("Saving result...")
    result.save(output_path)
    print("✅ Done with one image.\n")

def process_folder(folder_name):
    folder_path = os.path.join(base_dir, folder_name)
    if not os.path.isdir(folder_path) or folder_name == "inpainting_results":
        return None

    print(f"\n \U0001F4C2 Entering folder: {folder_name}")
    base_id = folder_name
    image_path = os.path.join(folder_path, f"{base_id}_orig.jpg")
    mask_path = os.path.join(folder_path, f"{base_id}_fake_mask.png")
    fake_path = os.path.join(folder_path, f"{base_id}_fake.jpg")

    if not (os.path.exists(image_path) and os.path.exists(mask_path) and os.path.exists(fake_path)):
        print(f"Skipping {base_id}: missing one of the required images.")
        return None

    metrics = {'filename': f"{base_id}_orig.jpg", 'base_id': base_id}
    reference_images = None

    for model_name, model_pipe in controlnet_pipes.items():
        output_path = os.path.join(output_dirs[model_name], f"{base_id}_result_{model_name}.jpg")
        try:
            run_controlnet_inpaint(
                image_path, mask_path, fake_path,
                model_pipe, reference_images, prompt, output_path, seed=42
            )

            gt_image = Image.open(image_path).convert("RGB")
            pred_image = Image.open(output_path).convert("RGB")
            psnr, ssim, lpips = evaluate_metrics(gt_image, pred_image)

            metrics[f'{model_name}_PSNR'] = psnr
            metrics[f'{model_name}_SSIM'] = ssim
            metrics[f'{model_name}_LPIPS'] = lpips

        except Exception as e:
            print(f"Error with {model_name} on {base_id}: {str(e)}")
            metrics[f'{model_name}_PSNR'] = -1
            metrics[f'{model_name}_SSIM'] = -1
            metrics[f'{model_name}_LPIPS'] = -1

    return metrics

In [ ]:
from huggingface_hub import login

if __name__ == "__main__":
    # First try to login to Hugging Face (optional)
    try:
        login(token="hf_YLBEIQWasZiRkWjteKqsSvEjQwtKwlYNoie")
    except:
        pass
    drive.mount('/content/drive', force_remount=True)

    base_dir = "/content/drive/MyDrive/dataset_controlnet"
    results_dir = os.path.join(base_dir, "inpainting_results")
    os.makedirs(results_dir, exist_ok=True)

    model_paths = {
        "v11p": "/content/drive/MyDrive/models/controlnet/inpaint",
        "v11f1p": "/content/drive/MyDrive/models/controlnet/seg",
        "v11canny": "/content/drive/MyDrive/models/controlnet/canny"
    }

    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting",
    )

    controlnet_pipes = {}
    for model_name, path in model_paths.items():
        controlnet = ControlNetModel.from_pretrained(path, local_files_only=True)
        controlnet_pipes[model_name] = StableDiffusionControlNetInpaintPipeline(
            vae=pipe.vae,
            text_encoder=pipe.text_encoder,
            tokenizer=pipe.tokenizer,
            unet=pipe.unet,
            scheduler=pipe.scheduler,
            safety_checker=None,
            feature_extractor=pipe.feature_extractor,
            controlnet=controlnet
        )

    output_dirs = {
        model_name: os.path.join(results_dir, f"controlnet_{model_name}")
        for model_name in controlnet_pipes
    }
    for d in output_dirs.values():
        os.makedirs(d, exist_ok=True)

    prompt = (
        "Replace the masked region with a natural extension of the surrounding background, "
        "ensuring the textures, colors, and lighting blend seamlessly. "
        "Do not recreate any specific object shapes from the mask."
    )

    evaluation_results = []
    selected_folders = sorted(os.listdir(base_dir))[:5]  # Only process first 5 folders

    with ThreadPoolExecutor(max_workers=3) as executor:
        for result in tqdm(executor.map(process_folder, selected_folders), total=len(selected_folders)):
            if result:
                evaluation_results.append(result)

    csv_file_path = os.path.join(results_dir, "evaluation_results.csv")
    if evaluation_results:
        fieldnames = ['filename', 'base_id'] + [k for k in evaluation_results[0] if k not in ['filename', 'base_id']]
        with open(csv_file_path, mode='w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(evaluation_results)

        print(f"\n✅ Done! Results saved to: {csv_file_path}")
    else:
        print("\n❌ No data was processed. Check folder or file naming.")
